In [1]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

device cuda:0
Using torch 2.0.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12287MB, multi_processor_count=28)


In [3]:

from torchvision import transforms
train_transform = transforms.Compose([
                                      transforms.Grayscale(1),
                                      transforms.ToTensor(),
                                     ])

test_transform = transforms.Compose([
                                     transforms.Grayscale(1),
                                     transforms.ToTensor(),
                                    ])

In [4]:
dataset_dir =  'C:/Users/liFangzheng/Desktop/LI Fangzheng/embryo_split'

In [5]:
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')

In [6]:
print('train_path', train_path)
print('test_path', test_path)

train_path C:/Users/liFangzheng/Desktop/LI Fangzheng/embryo_split\train
test_path C:/Users/liFangzheng/Desktop/LI Fangzheng/embryo_split\val


In [7]:
from torchvision import datasets

In [8]:
train_dataset = datasets.ImageFolder(train_path, train_transform)

In [9]:
test_dataset = datasets.ImageFolder(test_path, test_transform)

In [10]:
print('num_of_training_image', len(train_dataset))
print('num_of_classes', len(train_dataset.classes))
print('name_of_classes', train_dataset.classes)
print('num_of_test_image', len(test_dataset))
print('num_of_classes', len(test_dataset.classes))
print('name_of_classes', test_dataset.classes)

num_of_training_image 1880
num_of_classes 7
name_of_classes ['1.5fold', '2fold', 'defect', 'dorsal intercalation', 'gastrulation', 'rotation', 'ventral enclosure']
num_of_test_image 764
num_of_classes 7
name_of_classes ['1.5fold', '2fold', 'defect', 'dorsal intercalation', 'gastrulation', 'rotation', 'ventral enclosure']


In [11]:

class_names = train_dataset.classes
n_class = len(class_names)

In [12]:
n_class

7

In [13]:

train_dataset.class_to_idx

{'1.5fold': 0,
 '2fold': 1,
 'defect': 2,
 'dorsal intercalation': 3,
 'gastrulation': 4,
 'rotation': 5,
 'ventral enclosure': 6}

In [14]:
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [15]:
idx_to_labels

{0: '1.5fold',
 1: '2fold',
 2: 'defect',
 3: 'dorsal intercalation',
 4: 'gastrulation',
 5: 'rotation',
 6: 'ventral enclosure'}

In [16]:
np.save("C:/Users/liFangzheng/Desktop/LI Fangzheng/labels/idx_to_labels.npy",idx_to_labels)
np.save('C:/Users/liFangzheng/Desktop/LI Fangzheng/labels/idx_to_labels.npy', train_dataset.class_to_idx)

In [17]:
from torch.utils.data import DataLoader

In [18]:
BATCH_SIZE = 40


train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=0
                         )


test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=0
                        )

In [19]:
images, labels = next(iter(train_loader))

In [20]:
images.shape

torch.Size([40, 1, 256, 256])

In [21]:
labels

tensor([5, 5, 0, 0, 5, 3, 0, 6, 3, 1, 2, 6, 4, 6, 0, 4, 4, 5, 3, 1, 3, 2, 4, 6,
        4, 2, 1, 6, 2, 5, 1, 5, 4, 3, 5, 3, 6, 1, 5, 4])

In [22]:
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

In [23]:
model = models.resnet18(weights=None) 
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, n_class)
optimizer = optim.Adam(model.parameters())

In [24]:
model = model.to(device)


criterion = nn.CrossEntropyLoss() 


EPOCHS = 80


lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

In [25]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [26]:
epochs=[]
batch_num=[]


df_train_loss=[]
df_train_accuracy=[]


df_test_loss=[]
df_test_accuracy=[]
df_test_precision=[]
df_test_recall=[]
df_test_f1_score=[]

batch_idx = 0
best_test_accuracy = 0

In [27]:
def tarin_one_batch (images, labels):
   
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
   
    _,preds = torch.max(outputs,1)
    preds = preds.cpu().numpy()
    train_loss = loss.detach().cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()

    train_accuracy =accuracy_score(labels, preds)



    return train_loss, train_accuracy
    
    

In [28]:
def evaluate_testset():
    

    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) 

           
            _, preds = torch.max(outputs, 1) 
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) 
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)


    test_loss =  np.mean(loss_list)
    test_accuracy = accuracy_score(labels_list, preds_list)
    test_precision = precision_score(labels_list, preds_list, average='macro')
    test_recall =  recall_score(labels_list, preds_list, average='macro')
    test_f1_score = f1_score(labels_list, preds_list, average='macro')

    return test_loss, test_accuracy, test_precision, test_recall, test_f1_score

In [29]:
for epoch in range(EPOCHS):
    
    print(f'Epoch {epoch}/{EPOCHS}')
    
    
    model.train()
    for images, labels in tqdm(train_loader): 
        batch_idx += 1


        train_loss, train_accuracy = tarin_one_batch (images, labels)
       

        
    lr_scheduler.step()

   
    
   
    model.eval()
    test_loss, test_accuracy, test_precision, test_recall, test_f1_score = evaluate_testset()
    


    epochs.append(epoch)
    batch_num.append(batch_idx)

    
    df_train_loss.append(train_loss)
    df_train_accuracy.append(train_accuracy)


    df_test_loss.append(test_loss)
    df_test_accuracy.append(test_accuracy)
    df_test_precision.append(test_precision)
    df_test_f1_score.append(test_f1_score)


   
    if test_accuracy > best_test_accuracy: 
        
        old_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        
        best_test_accuracy = test_accuracy
        new_best_checkpoint_path = 'C:/Users/liFangzheng/Desktop/LI Fangzheng/save/best-{:.3f}.pth'.format(test_accuracy)
        torch.save(model, new_best_checkpoint_path)
        print('best_model', 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))
        



log_train = {}
log_train['epoch'] = epochs
log_train['train_loss'] = df_train_loss
log_train['train_accuracy'] = df_train_accuracy
df_train_log = pd.DataFrame(log_train)

df_train_log.to_csv('C:/Users/liFangzheng/Desktop/LI Fangzheng/data/train_data.csv', index=False)


log_test ={}
log_test['epoch'] = epochs
log_test['test_loss'] = df_test_loss
log_test['test_accuracy'] = df_test_accuracy
log_test['test_precision'] = df_test_precision
log_test['test_f1_score'] = df_test_f1_score
df_test_log = pd.DataFrame(log_test)
df_test_log.to_csv('C:/Users/liFangzheng/Desktop/LI Fangzheng/data/vali_data.csv', index=False)


Epoch 0/80


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:17<00:00,  2.67it/s]
D:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


best_model checkpoint/best-0.131.pth
Epoch 1/80


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:06<00:00,  7.15it/s]
D:\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/80


...

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:06<00:00,  7.30it/s]


Epoch 78/80


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:06<00:00,  7.29it/s]


Epoch 79/80


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:06<00:00,  7.32it/s]
